In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1650793755033_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

In [2]:
from pyspark.sql.functions import col
from pyspark.sql import *
from pyspark.sql.functions import count
from pyspark.sql.functions import when

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# load all the data (includes two springer tables and three AI tables after filtering) for one subfield. e.g. 0101-Pure Mathematics
def loaddata():
    springer_paper_path = "s3://cse6242-hchen703/0101_paper.csv"
    AI_path = "s3://cse6242-hchen703/AIpaperInfo.csv"
    springer_person_path = "s3://cse6242-hchen703/0101_paper_person.csv"
    AI_citation_path = "s3://cse6242-hchen703/citation.csv"
    AI_reference_path = "s3://cse6242-hchen703/reference.csv"
    springerdata = spark.read.csv(springer_paper_path, header=True, inferSchema=True)
    AIdata = spark.read.csv(AI_path, header=True, inferSchema=True)
    springerperson = spark.read.csv(springer_person_path, header=True, inferSchema=True)
    AIcitation = spark.read.csv(AI_citation_path, header=True, inferSchema=True)
    AIreference = spark.read.csv(AI_reference_path, header=True, inferSchema=True)
    return springerdata, AIdata, springerperson, AIcitation, AIreference

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
springerdata, AIdata, springerperson, AIcitation, AIreference= loaddata()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# join the rawdata of AI with filtered springer data
springerdata = springerdata.alias("springer")
AIdata = AIdata.alias("ai")
citation = springerdata.join(AIdata, col("springer.doi") == col("ai.doi")).select(col("springer.doi"),col("paper_name"),col("journal_id"), col("ai.AIid"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# join the rawdata of reference and citation from AI with filtered springer data
AIcitation = AIcitation.alias("AIcitation")
AIreference = AIreference.alias("AIreference")
citation = citation.alias("citation")
citationNeed = citation.join(AIcitation, col("citation.doi") == col("AIcitation.doi")).select(col("citation.doi"), col("cite_AIid"))
referenceNeed = citation.join(AIreference, col("citation.doi") == col("AIreference.doi")).select(col("citation.doi"), col("ref_AIid"))

#transform the AIid of reference paper and citation paper to doi
citationNeed = citationNeed.alias("cite_need")
referenceNeed = referenceNeed.alias("ref_need")
AIdata = AIdata.alias("ai")
ref_transform = referenceNeed.join(AIdata, col("ref_need.ref_AIid") == col("ai.AIid")).select(col("ref_need.doi"), col("ai.doi").alias("ref_doi"))
cite_transform = citationNeed.join(AIdata, col("cite_need.cite_AIid") == col("ai.AIid")).select(col("cite_need.doi"), col("ai.doi").alias("cite_doi"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# focus the citation and reference part to our subfield scale
ref_transform = ref_transform.alias("ref_t")
cite_transform = cite_transform.alias("cite_t")
springerdata = springerdata.alias("springer")
ref = ref_transform.join(springerdata, col("ref_t.ref_doi") == col("springer.doi")).select(col("ref_t.doi"), col("springer.doi").alias("ref_doi"))
cite = cite_transform.join(springerdata, col("cite_t.cite_doi") == col("springer.doi")).select(col("cite_t.doi"), col("springer.doi").alias("cite_doi"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# get paper-to-paper relationship and write the csv file
ref_reorder = ref.select(col("ref_doi").alias("doi"), col("doi").alias("cite_doi"))
relationship = ref_reorder.union(cite).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# add the journal information to the paper-to-paper relationship dataframe
springerdata = springerdata.alias("springer")
relationship = relationship.alias("relationship")
add_journal1 = relationship.join(springerdata, col("relationship.doi") == col("springer.doi")).select(col("relationship.doi").alias("doi"), col("springer.journal_id").alias("journal_id"), col("relationship.cite_doi"))
add_journal1 = add_journal1.alias("add_journal1")
add_journal2 = add_journal1.join(springerdata, col("add_journal1.cite_doi") == col("springer.doi")).select(col("add_journal1.journal_id").alias("journal_id"), col("springer.journal_id").alias("cite_journal_id"))
journal_relationship = add_journal2.groupBy(col("journal_id"), col("cite_journal_id")).agg(count("*").alias("cite_num"))
journal = journal_relationship.filter(journal_relationship.journal_id.like('sg%')).filter(journal_relationship.cite_journal_id.like('sg%'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# filter journal-to-journal dataframe to those with clear snip and sjr info and get the final journal-to-journal relationship csv file
raw_journal = spark.read.csv("s3://cse6242-hchen703/6242projetc_journal/journal.csv", header=True, inferSchema=True)
raw_journal = raw_journal.alias("raw")
journal = journal.alias("journal")
journal_filter1 = journal.join(raw_journal, col("journal.journal_id") == col("raw.journal_id")).select(col("raw.journal_id"), col("journal.cite_journal_id"), col("journal.cite_num"))
journal_filter1 = journal_filter1.alias("filter")
journal_filter2 = journal_filter1.join(raw_journal, col("filter.cite_journal_id") == col("raw.journal_id")).select(col("filter.journal_id"), col("filter.cite_journal_id"), col("filter.cite_num"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# add author information to the dataframe of paper relationship and compute the sum of lambda valve for per author per paper
person = person.alias("person")
relationship = relationship.alias("relationship")
add_person1 = relationship.join(person, col("relationship.doi") == col("person.doi")).select(col("relationship.doi"), col("person.author_id"), col("relationship.cite_doi"))
temp = add_person1.groupBy(col("doi"), col("author_id")).agg(count("*").alias("cite_num"))
add_person1 = add_person1.alias("add_person")
add_person2 = add_person1.join(person, col("add_person.cite_doi") == col("person.doi")).select(col("add_person.doi"), col("add_person.author_id"), col("person.author_id").alias("cite_author_id"))
temp2 = add_person2.withColumn("selfcite", when(add_person2.author_id == add_person2.cite_author_id, 1).otherwise(0))
temp3 = temp2.select("doi", "author_id", "selfcite")
temp4 = temp3.groupBy(col("doi"), col("author_id")).agg({"selfcite": "sum"}).withColumnRenamed("sum(selfcite)", "selfcite_num")
temp = temp.alias("temp")
temp4 = temp4.alias("temp4")
lambda = temp4.join(temp, (col("temp4.doi") == col("temp.doi")) & (col("temp4.author_id") == col("temp.author_id"))).withColumn("sum_lambda", temp["cite_num"] - 0.5 * temp4["selfcite_num"]).select(col("temp.doi"), col("temp.author_id"), col("sum_lambda"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
from pyspark.sql.types import IntegerType
# load the longest path data for papers after building the directed paper citation graph 
paper_path = spark.read.csv("s3://cse6242-hchen703/paper_path/0101path.csv", header=True, inferSchema=True)
paper_path.withColumn("LonestPath",paper_path.LonestPath.cast(IntegerType()))
paper_weight = paper_path.select("Source", "LonestPath").distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Source: string (nullable = true)
 |-- LonestPath: integer (nullable = true)

+--------------------+----------+
|              Source|LonestPath|
+--------------------+----------+
|10.1007/s11225-00...|         2|
|10.1007/s10623-01...|         3|
|10.1007/s00220-02...|         2|
|10.1007/s11784-01...|         1|
|10.1007/s00209-01...|         1|
|10.1007/s00220-00...|         1|
|10.1007/s00021-02...|         2|
|10.1007/s13348-01...|         1|
+--------------------+----------+
only showing top 8 rows

In [23]:
from pyspark.sql.functions import *
from pyspark.sql.functions import round
from decimal import Decimal
# join the paper influence ratio of the specific subfield with the author lambda values
maxpath = paper_weight.agg({"LonestPath": "max"}).collect()[0][0]
print(maxpath)
I = lamda.join(paper_weight, col("doi") == col("Source")).select(col("doi"), col("LonestPath"), col("author_id"), col("sum_lamda")).withColumn("ratio", col("LonestPath")/maxpath).withColumn("influence", col("ratio")*col("sum_lamda"))

author_score = I.groupBy("author_id").agg({"influence": "sum"}).withColumn("field", lit("Pure Mathematics")).withColumnRenamed("sum(influence)", "score")
author_IF = author_score.select("author_id", "score", "field")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

49
root
 |-- doi: string (nullable = true)
 |-- LonestPath: integer (nullable = true)
 |-- author_id: string (nullable = true)
 |-- sum_lamda: double (nullable = true)
 |-- ratio: double (nullable = true)
 |-- influence: double (nullable = true)

In [26]:
from pyspark.sql.types import DoubleType
# load the full jorunal influence number csv file and compute the average influence number for each journal that from different subfields
journal_score = spark.read.csv("s3://cse6242-hchen703/project_output/journal_IN.csv", header=True, inferSchema=True)
journal_change_type = journal_score.withColumn("score",journal_score.score.cast('double'))
journal_IN = journal_change_type.groupBy("journal_id").agg({"score":"avg"})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- journal_id: string (nullable = true)
 |-- avg(score): double (nullable = true)

In [27]:
add_person1 = add_person1.alias("add_person1")
springerdata = springerdata.alias("springer")
journal_paper_author = add_person1.join(springerdata, col("add_person1.doi") == col("springer.doi")).select(col("springer.doi"), col("springer.journal_id"), col("add_person1.author_id"))
journal_IN = journal_IN.alias("IN") 
journal_paper_author = journal_paper_author.alias("jpa")
add_journal_score = journal_paper_author.join(journal_IN, col("jpa.journal_id") == col("IN.journal_id")).select(col("jpa.journal_id"), col("IN.avg(score)").alias("journal_IN"), col("jpa.author_id")).groupBy("author_id").agg({"journal_IN": "sum"})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- author_id: string (nullable = true)
 |-- sum(journal_IN): double (nullable = true)

In [ ]:
# compute the final score of authors and write into the author_score table to be used in our database
author_IF = author_IF.alias("IF")
add_journal_score = add_journal_score.alias("journal_score")
finalscore_temp =  author_IF.join(add_journal_score, col("IF.author_id") == col("journal_score.author_id")).select(col("IF.author_id"), col("IF.score").alias("author_IF"), col("journal_score.sum(journal_IN)").alias("journal_IN"), col("IF.field")).withColumn("score", col("author_IF")+col("journal_IN"))
author_finalscore = finalscore_temp.select(col("author_id"), round(col("score"),4).alias("score"), col("field"))
author_finalscore.coalesce(1).write.option("header","false").csv("s3://cse6242-hchen703/project_database/0101author_score")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# join the author info to the author_score dataframe and get the author table for our database
author_finalscore = author_finalscore.alias("score")
person = person.alias("person")
author = author_finalscore.join(person, col("person.author_id") == col("score.author_id")).select(col("person.author_id"), col("person.author_name"), col("person.affiliation_id")).distinct()
author.printSchema()
author.coalesce(1).write.option("header","false").csv("s3://cse6242-hchen703/project_database/0101author")

In [ ]:
# join all the publication info to the author table and get the author _publication table for our database
person = person.alias("person")
author_finalscore = author_finalscore.alias("score")
author_publication = author_finalscore.join(person, col("person.author_id") == col("score.author_id")).select(col("score.author_id"), col("person.doi"))
author_publication.printSchema()
author_publication.coalesce(1).write.option("header","false").csv("s3://cse6242-hchen703/project_database/0101author_publication")

In [ ]:
# join all the #citation info for each paper and get the articles table for our database
springerdata = springerdata.alias("springer")
AIdata = AIdata.alias("ai")
paper = springerdata.join(AIdata, col("springer.doi") == col("ai.doi")).select(col("springer.doi"),col("paper_name"),col("cite_num"))
articles = paper_weight.join(paper, col("Source") == col("doi")).select(col("doi"), col("paper_name"), col("cite_num"))
articles.printSchema()
articles.coalesce(1).write.option("header","false").csv("s3://cse6242-hchen703/project_database/0101articles")